In [ ]:
ls yelp_data/

In [1]:
import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim
import os, re, operator, warnings
warnings.filterwarnings('ignore') # Let's not pay heed to them right now
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np

In [3]:
path = 'yelp_data/health_text_sentiment_full.csv'

In [22]:
dft = pd.read_csv(path)
dft.drop(['Unnamed: 0'],axis=1,inplace=True)
#df = df[['stars','text']]
dft = dft[['business_id','text','clean_text']]

In [23]:
dft.head()

,business_id,text,clean_text
0,y-4xTZNKVm8mAZpiXMS5ZA,Please stay away from this place if you can! I...,please stay away place bad care imaginable sta...
1,y-4xTZNKVm8mAZpiXMS5ZA,My husband has been a patient of Dr. Byrne for...,husband patient dr byrne last year half last m...
2,y-4xTZNKVm8mAZpiXMS5ZA,Dr. Byrne is a great doctor! She has great bed...,dr byrne great doctor great bed side manner ex...
3,y-4xTZNKVm8mAZpiXMS5ZA,I'm raising my review as Dr Bryne's has been m...,raise review dr bryne receptive daughter go an...
4,y-4xTZNKVm8mAZpiXMS5ZA,I wish I could give 0 stars. Worst office I've...,wish could give star bad office ever horrible ...


In [6]:
%%time
import spacy
nlp = spacy.load('en_core_web_lg')

## grab sample review

In [24]:
sample = dft.text[10090]
sample

"I was referred here by a friend who told me she had received wonderful care by Dr. Leonard. My first and only appointment was terrible. The front staff were very warm and friendly with me. I had just suffered my 2nd miscarriage, was still dealing with my mother's recent death, and was caring for my parents' young adopted children. I had been divorced and received an annulment through the Catholic Church. My new husband and I had been married for 2 years and, dealing with all of the previously mentioned problems, we had not had time to finish the process of getting our marriage blessed in the church. I went in to consult with Dr. Leonard. First, we commented rudely about me being slightly overweight. Then, he made another rude comment about me smelling like smoke and he didn't want to treat me until I had quit smoking. Then, he said he would not begin treating me until my husband and I had our marriage blessed in the church. I explained my family situation and why we had not gotten our

## parse text

In [8]:
%%time
parsed_rev = nlp(sample)

CPU times: user 672 ms, sys: 46.9 ms, total: 719 ms
Wall time: 112 ms


In [9]:
sample

"I was referred here by a friend who told me she had received wonderful care by Dr. Leonard. My first and only appointment was terrible. The front staff were very warm and friendly with me. I had just suffered my 2nd miscarriage, was still dealing with my mother's recent death, and was caring for my parents' young adopted children. I had been divorced and received an annulment through the Catholic Church. My new husband and I had been married for 2 years and, dealing with all of the previously mentioned problems, we had not had time to finish the process of getting our marriage blessed in the church. I went in to consult with Dr. Leonard. First, we commented rudely about me being slightly overweight. Then, he made another rude comment about me smelling like smoke and he didn't want to treat me until I had quit smoking. Then, he said he would not begin treating me until my husband and I had our marriage blessed in the church. I explained my family situation and why we had not gotten our

In [10]:
parsed_rev

I was referred here by a friend who told me she had received wonderful care by Dr. Leonard. My first and only appointment was terrible. The front staff were very warm and friendly with me. I had just suffered my 2nd miscarriage, was still dealing with my mother's recent death, and was caring for my parents' young adopted children. I had been divorced and received an annulment through the Catholic Church. My new husband and I had been married for 2 years and, dealing with all of the previously mentioned problems, we had not had time to finish the process of getting our marriage blessed in the church. I went in to consult with Dr. Leonard. First, we commented rudely about me being slightly overweight. Then, he made another rude comment about me smelling like smoke and he didn't want to treat me until I had quit smoking. Then, he said he would not begin treating me until my husband and I had our marriage blessed in the church. I explained my family situation and why we had not gotten our 

## spacy attributes

### Sentensce detetcion ans segmentation

In [11]:
for num, sentence in enumerate(parsed_rev.sents):
    print('Sentence {}:\n{}'.format(num+1,sentence))

Sentence 1:
I was referred here by a friend who told me she had received wonderful care by Dr. Leonard.
Sentence 2:
My first and only appointment was terrible.
Sentence 3:
The front staff were very warm and friendly with me.
Sentence 4:
I had just suffered my 2nd miscarriage, was still dealing with my mother's recent death, and was caring for my parents' young adopted children.
Sentence 5:
I had been divorced and received an annulment through the Catholic Church.
Sentence 6:
My new husband and I had been married for 2 years and, dealing with all of the previously mentioned problems, we had not had time to finish the process of getting our marriage blessed in the church.
Sentence 7:
I went in to consult with Dr. Leonard.
Sentence 8:
First, we commented rudely about me being slightly overweight.
Sentence 9:
Then, he made another rude comment about me smelling like smoke and he didn't want to treat me until I had quit smoking.
Sentence 10:
Then, he said he would not begin treating me unti

### Entities

In [12]:
for num, entity in enumerate(parsed_rev.ents):
    print('Entity {}: {} - {}'.format(num+1,entity,entity.label_))

Entity 1: Leonard - PERSON
Entity 2: first - ORDINAL
Entity 3: 2nd - ORDINAL
Entity 4: the Catholic Church - ORG
Entity 5: 2 years - DATE
Entity 6: Leonard - PERSON
Entity 7: First - ORDINAL
Entity 8: Catholic - NORP
Entity 9: Catholic - NORP
Entity 10: MD - GPE
Entity 11: Leonard - PERSON


## Part of speach tagging

In [13]:
token_text = [token.orth_ for token in parsed_rev]
token_pos = [token.pos_ for token in parsed_rev]

pd.DataFrame(list(zip(token_text,token_pos)),columns=['token_text','POS'])

,token_text,POS
0,I,PRON
1,was,VERB
2,referred,VERB
3,here,ADV
4,by,ADP
5,a,DET
6,friend,NOUN
7,who,NOUN
8,told,VERB
9,me,PRON


## Normalization 
- stemming  
- lemmatitations (normalized word)
- shape analysis

In [14]:
token_lemma = [token.lemma_ for token in parsed_rev]
token_shape = [token.shape_ for token in parsed_rev]

pd.DataFrame(list(zip(token_text,token_lemma,token_shape)),columns=['token_text','token_lemma','token_shape'])

,token_text,token_lemma,token_shape
0,I,-PRON-,X
1,was,be,xxx
2,referred,refer,xxxx
3,here,here,xxxx
4,by,by,xx
5,a,a,x
6,friend,friend,xxxx
7,who,who,xxx
8,told,tell,xxxx
9,me,-PRON-,xx


## Token level entitiy anbalysis

In [15]:
token_entity_type = [token.ent_type for token in parsed_rev]
token_entity_iob = [token.ent_iob_ for token in parsed_rev]

pd.DataFrame(list(zip(token_text,token_entity_type,token_entity_iob)),columns=['token_text','entity_type','inside_outside_beguin'])

,token_text,entity_type,inside_outside_beguin
0,I,0,O
1,was,0,O
2,referred,0,O
3,here,0,O
4,by,0,O
5,a,0,O
6,friend,0,O
7,who,0,O
8,told,0,O
9,me,0,O


## variety of token-level attributes

- stopwords
- punctuation
- whitespace
- represents a num
- token in spacy default voc?

In [16]:
token_att = [(token.orth_,
              token.prob,
              token.is_stop,
              token.is_punct,
              token.is_space,
              token.like_num,
              token.is_oov) for token in parsed_rev]

df = pd.DataFrame(token_att, 
                  columns=['text',
                           'log_prob',
                           'stop',
                           'piunctuation?',
                         'whitespace',
                         'num?',
                         'out_of_vocab'])

df.loc[:,'stop':'out_of_vocab'] = (df.loc[:,'stop':'out_of_vocab'].applymap(lambda x: 'Yes' if x else ' '))



In [17]:
df

,text,log_prob,stop,piunctuation?,whitespace,num?,out_of_vocab
0,I,-3.791565,,,,,
1,was,-5.252320,,,,,
2,referred,-11.325837,,,,,
3,here,-7.068025,,,,,
4,by,-6.375087,,,,,
5,a,-3.929788,,,,,
6,friend,-8.210516,,,,,
7,who,-6.504637,,,,,
8,told,-8.523034,,,,,
9,me,-5.846090,,,,,


In [18]:
df.log_prob.describe()

count    294.000000
mean      -7.049900
std        2.930240
min      -20.000000
25%       -9.473791
50%       -6.454271
75%       -4.619072
max       -3.067898
Name: log_prob, dtype: float64

## Topic modeling

# Topic Modeling with Latent Dirichlet Allocation (LDA)


In [87]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle as pickle

In [91]:
aa = dft.clean_text[0]

In [92]:
LineSentence(aa)

In [ ]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if 1== 1:

    trigram_reviews = LineSentence(aa)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)